In [1]:
import numpy as np
import pandas as pd
from utils import config, database_service

In [2]:
datalake_db = database_service.DataService(config.get_config()['datalake_write'])

## SG BTL Loan Universe

In [3]:
sg_btl = datalake_db.query_postgres('''
    SELECT 
        id AS loan_id,
        loan_code,
        amount,
        interest_rate,
        interest_type,
        tenor,
        tenor_type
    FROM loan_db.loans
    WHERE country_code = 'SG'
    AND product_id IN (8, 10)
    AND status LIKE '%%SET%%'
''')

In [4]:
print('*There are a total of {} disbursed SG BTL loans.*'.format(sg_btl.shape[0]))

*There are a total of 431 disbursed SG BTL loans.*


## SG Financial Statements

In [5]:
sg_balance_sheet = datalake_db.query_postgres('''
    SELECT DISTINCT ON (a.loan_code, a.date, a.level_1, a.level_2, a.level_3)
        b.id AS loan_id,
        'CREDIT SCORECARD' AS source
    FROM credit_scorecard.tbl_sg_balance_sheet a
    INNER JOIN (
        SELECT * 
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_code=b.loan_code
    ORDER BY a.loan_code, a.date, a.level_1, a.level_2, a.level_3, a.created_time DESC
''')

sg_income_statement = datalake_db.query_postgres('''
    SELECT DISTINCT ON (a.loan_code, a.date, a.level_1, a.level_2)
        b.id AS loan_id,
        'CREDIT SCORECARD' AS source
    FROM credit_scorecard.tbl_sg_income_statement a
    INNER JOIN (
        SELECT * 
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_code=b.loan_code
    ORDER BY a.loan_code, a.date, a.level_1, a.level_2, a.created_time DESC
''')

In [6]:
print('*There are a total of {} parsed SG balance sheets.*'.format(len(sg_balance_sheet['loan_id'].unique())))
print('*There are a total of {} parsed SG income statements.*'.format(len(sg_income_statement['loan_id'].unique())))

*There are a total of 215 parsed SG balance sheets.*
*There are a total of 217 parsed SG income statements.*


## SG Bank Statements

In [7]:
sg_bank_statements_ocr = datalake_db.query_postgres('''
    SELECT DISTINCT 
        a.loan_id,
        'OCR' AS source
    FROM loan_documents_data_db.loans_to_bank_accounts a
    INNER JOIN (
        SELECT * 
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
''')

sg_bank_statements_credit_scorecard = datalake_db.query_postgres('''
    SELECT DISTINCT 
        b.id AS loan_id,
        'CREDIT SCORECARD' AS source
    FROM credit_scorecard.tbl_sg_bank_analysis a
    INNER JOIN (
        SELECT * 
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_code=b.loan_code
''')


sg_bank_statements_microloan = datalake_db.query_postgres('''
    SELECT DISTINCT ON (a.loan_id, a.bank_account_ref_id)
        a.loan_id,
        'MICROLOAN DB' AS source
    FROM microloan.bank_statement_details a
    INNER JOIN (
        SELECT * 
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
    ORDER BY a.loan_id, a.bank_account_ref_id, created_time DESC
''')

sg_bank_statements = pd.concat([sg_bank_statements_ocr,
                                sg_bank_statements_credit_scorecard,
                                sg_bank_statements_microloan])

In [8]:
print('*There are a total of {} parsed SG bank statements.*'.format(sg_bank_statements.shape[0]))
print('There are a total of {} parsed SG Internal OCR bank statements.'.format(sg_bank_statements_ocr.shape[0]))
print('There are a total of {} parsed SG Perfios bank statements.'.format(sg_bank_statements_credit_scorecard.shape[0]))
print('There are a total of {} parsed SG Microloan bank statements.'.format(sg_bank_statements_microloan.shape[0]))

*There are a total of 182 parsed SG bank statements.*
There are a total of 67 parsed SG Internal OCR bank statements.
There are a total of 56 parsed SG Perfios bank statements.
There are a total of 59 parsed SG Microloan bank statements.


## SG Credit Documents CBS

In [9]:
sg_cbs_loan_doc_db = datalake_db.query_postgres('''
    SELECT 
        a.loan_id,
        'LOAN DOC DB' AS source
    FROM loan_documents_data_db.loans_to_cbs a
    INNER JOIN (
        SELECT *
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
''')

sg_cbs_dms = datalake_db.query_postgres('''
    SELECT DISTINCT ON (loan_id)
        a.loan_id,
        'DMS DB' AS source
    FROM dms.loan_doc a
    INNER JOIN (
        SELECT *
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
    WHERE a.type='CBS'
    AND a.json <> ''
    AND a.json IS NOT NULL
    ORDER BY a.loan_id, a.created_time DESC
''')

sg_cbs = sg_cbs_loan_doc_db.append(sg_cbs_dms, ignore_index=True)

In [10]:
print('*There are a total of {} parsed SG CBS documents*'.format(len(sg_cbs['loan_id'].unique())))
print('There are a total of {} parsed SG Loan Doc DB CBS documents.'.format(sg_cbs_loan_doc_db.shape[0]))
print('There are a total of {} parsed SG DMS CBS documents.'.format(sg_cbs_dms.shape[0]))

*There are a total of 200 parsed SG CBS documents*
There are a total of 195 parsed SG Loan Doc DB CBS documents.
There are a total of 186 parsed SG DMS CBS documents.


## SG Credit Documents BRI

In [11]:
sg_bri = datalake_db.query_postgres('''
    SELECT DISTINCT ON (loan_id)
        a.loan_id,
        'DMS DB' AS source
    FROM dms.loan_doc a
    INNER JOIN (
        SELECT *
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
    WHERE a.type LIKE '%%BRI%%'
    AND a.json <> ''
    AND a.json IS NOT NULL
    ORDER BY a.loan_id, a.created_time DESC
''')

sg_bri_html = datalake_db.query_postgres('''
    SELECT DISTINCT ON (loan_id)
        a.loan_id,
        'DMS DB' AS source
    FROM dms.loan_doc a
    INNER JOIN (
        SELECT *
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
    WHERE a.type = 'BRI_HTML'
    AND a.json <> ''
    AND a.json IS NOT NULL
    ORDER BY a.loan_id, a.created_time DESC
''')

In [12]:
print('*There are a total of {} parsed SG BRI documents*'.format(len(sg_bri['loan_id'].unique())))
print('There are a total of {} parsed SG BRI HTML documents'.format(len(sg_bri_html['loan_id'].unique())))

*There are a total of 242 parsed SG BRI documents*
There are a total of 12 parsed SG BRI HTML documents


## SG Credit Documents BRC

In [13]:
sg_brc = datalake_db.query_postgres('''
    SELECT DISTINCT ON (loan_id)
        a.loan_id,
        'DMS DB' AS source
    FROM dms.loan_doc a
    INNER JOIN (
        SELECT *
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
    WHERE a.type LIKE '%%BRC%%'
    AND a.json <> ''
    AND a.json IS NOT NULL
    ORDER BY a.loan_id, a.created_time DESC
''')

sg_brc_html = datalake_db.query_postgres('''
    SELECT DISTINCT ON (loan_id)
        a.loan_id,
        'DMS DB' AS source
    FROM dms.loan_doc a
    INNER JOIN (
        SELECT *
        FROM loan_db.loans
        WHERE country_code = 'SG'
        AND product_id IN (8, 10)
        AND status LIKE '%%SET%%') b ON a.loan_id=b.id
    WHERE a.type = 'BRC_HTML'
    AND a.json <> ''
    AND a.json IS NOT NULL
    ORDER BY a.loan_id, a.created_time DESC
''')

In [14]:
print('*There are a total of {} parsed SG BRC documents*'.format(len(sg_brc['loan_id'].unique())))
print('*There are a total of {} parsed SG BRC HTML documents*'.format(len(sg_brc_html['loan_id'].unique())))

*There are a total of 276 parsed SG BRC documents*
*There are a total of 14 parsed SG BRC HTML documents*


## SG BTL Data Quality Overview

In [15]:
sg_btl['financials_balance_sheet'] = np.where(sg_btl['loan_id'].isin(sg_balance_sheet['loan_id'].unique()), 1, 0)
sg_btl['financials_income_statement'] = np.where(sg_btl['loan_id'].isin(sg_income_statement['loan_id'].unique()), 1, 0)
sg_btl['bank_statements'] = np.where(sg_btl['loan_id'].isin(sg_bank_statements['loan_id'].unique()), 1, 0)
sg_btl['credit_doc_cbs'] = np.where(sg_btl['loan_id'].isin(sg_cbs['loan_id'].unique()), 1, 0)
sg_btl['credit_doc_brc'] = np.where(sg_btl['loan_id'].isin(sg_brc['loan_id'].unique()), 1, 0)
sg_btl['credit_doc_bri'] = np.where(sg_btl['loan_id'].isin(sg_bri['loan_id'].unique()), 1, 0)

In [16]:
sg_btl_2 = sg_btl[['loan_id',
                   'financials_balance_sheet',
                   'financials_income_statement',
                   'bank_statements',
                   'credit_doc_cbs',
                   'credit_doc_brc',
                   'credit_doc_bri']].dropna(axis=0).copy()

sg_btl_2['total'] = sg_btl[['financials_balance_sheet',
                            'financials_income_statement',
                            'bank_statements',
                            'credit_doc_cbs',
                            'credit_doc_brc',
                            'credit_doc_bri']].sum(axis=1)

In [17]:
for i in range(0, 7):
    print(sg_btl_2[sg_btl_2['total']==i].shape[0])

139
14
27
43
34
73
101
